In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import os

In [2]:
'''
Read in saved dataset to use to feed into the model.
Further pre-processing still needs to be done on the state vectors
'''

data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/ball_carrier_dataset.pkl')


data_df.head()

,season,gameId,playId,playIndex,timeIndex,time,playerId,football_pos,state,next_state,reward,action,next_action
0,2018,2018090600,677,0,0,2018-09-07T01:34:34.500,44979.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[11.53, 42.45, 0.14, 1.32, 3.03, 117.3, 148.0...","[[11.62, 42.32, 0.16, 1.7, 3.13, 114.42, 144.8...",[-0.09],"[[-0.09, 0.13, -0.02, -0.38, -0.1, 2.88, 3.22,...","[[-0.16, 0.16, -0.07, -0.53, -0.24, -17.77, 7...."
1,2018,2018090600,677,0,1,2018-09-07T01:34:34.600,44979.0,"[[11.51, 42.1, 0.39, 1.64, 4.44, 0.0, 0.0, 11....","[[11.62, 42.32, 0.16, 1.7, 3.13, 114.42, 144.8...","[[11.78, 42.16, 0.23, 2.23, 3.37, 132.19, 137....",[-0.16],"[[-0.16, 0.16, -0.07, -0.53, -0.24, -17.77, 7....","[[-0.17, 0.18, -0.01, -0.4, -0.51, 6.53, 2.66,..."
2,2018,2018090600,677,0,2,2018-09-07T01:34:34.700,44979.0,"[[11.47, 42.06, 0.06, 1.2, 3.4, 0.0, 0.0, 11.4...","[[11.78, 42.16, 0.23, 2.23, 3.37, 132.19, 137....","[[11.95, 41.98, 0.24, 2.63, 3.88, 125.66, 134....",[-0.17],"[[-0.17, 0.18, -0.01, -0.4, -0.51, 6.53, 2.66,...","[[-0.21, 0.19, -0.05, -0.48, -0.22, 4.32, 3.47..."
3,2018,2018090600,677,0,3,2018-09-07T01:34:34.800,44979.0,"[[12.39, 41.68, 1.0, 0.93, 2.17, 0.0, 0.0, 12....","[[11.95, 41.98, 0.24, 2.63, 3.88, 125.66, 134....","[[12.16, 41.79, 0.29, 3.11, 4.1, 121.34, 131.2...",[-0.21],"[[-0.21, 0.19, -0.05, -0.48, -0.22, 4.32, 3.47...","[[-0.25, 0.21, -0.03, -0.44, -0.67, 2.49, 3.83..."
4,2018,2018090600,677,0,4,2018-09-07T01:34:34.900,44979.0,"[[13.5, 41.18, 1.22, 1.47, 3.5, 0.0, 0.0, 13.5...","[[12.16, 41.79, 0.29, 3.11, 4.1, 121.34, 131.2...","[[12.41, 41.58, 0.32, 3.55, 4.77, 118.85, 127....",[-0.25],"[[-0.25, 0.21, -0.03, -0.44, -0.67, 2.49, 3.83...","[[-0.32, 0.21, -0.06, -0.47, -0.03, 0.0, 9.25,..."


In [3]:
'''
State vector 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''

data_df.loc[0,'state'][0]

array([ 1.15300000e+01,  4.24500000e+01,  1.40000000e-01,  1.32000000e+00,
        3.03000000e+00,  1.17300000e+02,  1.48060000e+02,  1.15300000e+01,
        1.58000000e+01,  2.73000000e+01,  5.80600000e+01,  8.27327901e-01,
       -5.61719276e-01,  9.98232532e-01,  5.94290537e-02,  0.00000000e+00,
        1.02310000e+01,  3.04200000e+01])

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [5]:
'''
Create torch dataset
'''

features = torch.zeros(len(data_df)).to(device)
labels = torch.zeros(len(data_df)).to(device)

print("size = ", features.size())

size =  torch.Size([8873])


In [6]:
'''
Function that splits the data into a training, validation, and test set
'''
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)

print(test_data.columns)

print(f"Length of training data: {len(train_data)}")

Index(['season', 'gameId', 'playId', 'playIndex', 'timeIndex', 'time',
       'playerId', 'football_pos', 'state', 'next_state', 'reward', 'action',
       'next_action'],
      dtype='object')
Length of training data: 6211


In [7]:
'''
Predict ['s', 'a', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']
from 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''



'''
Define hyperparamters
'''
BATCH_SIZE = 64
state_size = len(data_df.loc[0,'state'][0])
action_size = 8
gamma = 0.99
#max_action = torch.tensor(max_action, dtype=torch.float32).to(device=device)

print(f"State size: {state_size}")
print(f"Action size: {action_size}")
    
'''
First CNN model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Conv2d(12, 24, (6, 1)),
    nn.LeakyReLU(),
    nn.Conv2d(24, 48, (1,1)),
    nn.Flatten(),
    nn.Linear(48*6*6,action_size)
)

cnn_model.to(device=device)
'''

State size: 18
Action size: 8


'\nFirst CNN model\ncnn_model = nn.Sequential(\n    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),\n    nn.LeakyReLU(),\n    nn.Conv2d(12, 24, (6, 1)),\n    nn.LeakyReLU(),\n    nn.Conv2d(24, 48, (1,1)),\n    nn.Flatten(),\n    nn.Linear(48*6*6,action_size)\n)\n\ncnn_model.to(device=device)\n'

In [8]:
'''
cnn_model = nn.Sequential(
    nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(64,128, (2,2), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(128,256, (1,1), stride=1,padding=0),
    #nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Flatten(),
    nn.Linear(4352, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 100),
    nn.LeakyReLU(),
    nn.Linear(100, action_size)
)
cnn_model.to(device=device)
'''

# 0.3046 val accuracy after 4 epochs, batch=64
cnn_model = nn.Sequential(
    nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
    nn.Flatten(),
    nn.Linear(924,300), 
    nn.LeakyReLU(),
    nn.Linear(300,100), 
    nn.LeakyReLU(),
    nn.Linear(100,action_size)
)


'''
cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,100), 
    nn.LeakyReLU(),
    nn.Linear(100,50), 
    nn.LeakyReLU(),
    nn.Linear(50,action_size)
)

cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,60), 
    nn.Dropout(p=0.5),
    nn.LeakyReLU(),
    nn.Linear(60,action_size)
)
'''


cnn_model.to(device=device)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(2, 2), stride=(1, 1))
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=924, out_features=300, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=300, out_features=100, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=100, out_features=8, bias=True)
)

In [9]:
# define loss function
mse_loss_fn = nn.MSELoss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters())

In [10]:
np.stack(test_data.iloc[0,:]['action'])[0,1:3]

array([ 0.04, -0.02])

In [11]:
'''
Given indices, train model on batch
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''
def train_loop(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    numpy_state_input = numpy_state.reshape(batch_size, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
    
    true_state = torch.tensor(numpy_state, dtype=torch.float32).to(device=device)
    true_action = torch.tensor(np.stack(play['action'].values)[:,0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    #true_state_mod = true_state.reshape(batch_size, -1, len(true_action))
    true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_action = model(true_state_mod)
    
    loss = loss_fn(predicted_action, true_action)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [12]:
def test_loop(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            true_state = torch.tensor(play['state'], dtype=torch.float32).to(device=device)
            true_action = torch.tensor(play['action'][0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
            
            # useful for if testing in batches
            #true_state = torch.tensor(np.stack(play['state'].values), dtype=torch.float32).to(device=device)
            #true_action = torch.tensor(np.stack(play['action'].values), dtype=torch.float32).to(device=device)

            numpy_state_input = play['state'].reshape(1, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
                           
            true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_action = model(true_state_mod)
  
            true_action = torch.unsqueeze(true_action, 0)

            loss = loss_fn(predicted_action, true_action)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [13]:
import random

epochs = 4
break_var = False
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/2) == 0):# and (i != 0):
            val_loss = test_loop(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
            
torch.save(cnn_model.state_dict(), 'ball_carrier_model.pt')

Num batches: 98
At epoch 0, iter 0: train loss = 0.027493393048644066
At epoch 0, iter 0: val loss = 2.203003168106079
At epoch 0, iter 49: train loss = 0.00897911936044693
At epoch 0, iter 49: val loss = 0.6155802011489868
At epoch 1, iter 0: train loss = 0.00839539710432291
At epoch 1, iter 0: val loss = 0.558029055595398
At epoch 1, iter 49: train loss = 0.008281496353447437
At epoch 1, iter 49: val loss = 0.5514553189277649
At epoch 2, iter 0: train loss = 0.008095143362879753
At epoch 2, iter 0: val loss = 0.6300866603851318
At epoch 2, iter 49: train loss = 0.007830221205949783
At epoch 2, iter 49: val loss = 0.5414954423904419
At epoch 3, iter 0: train loss = 0.009310181252658367
At epoch 3, iter 0: val loss = 0.5533040761947632
At epoch 3, iter 49: train loss = 0.007571555208414793
At epoch 3, iter 49: val loss = 0.5101901292800903
